# Psycopg

Psycopg is a postgres adapter for python programming language.

In [4]:
import time
import docker
import psycopg
from utils.rerun_docker import reload_docker_container

client = docker.from_env()
container_name = "psycopg_examples"
container_port = 5432

container = reload_docker_container(
    container_name,
    image="postgres:17.4",
    environment={
        "POSTGRES_PASSWORD": "password",
    },
    ports={f"{container_port}/tcp": 5432},
    detach=True,
    remove=True
)

## Load data

To run anything in postgres you need to:

- Create a connection: `psycopg.connect()`.
- Create cursor: `connection.cursor()`.
- Execute your query: `cursor.execute()`.
- Get data from cursor: `cursor.fetchall`, `cursor.fetachone`, `cursor.description` etc.

---

The following cell connects to the database.

In [3]:
connection = psycopg.connect(
    dbname="postgres",
    user="postgres",
    password="password",
    host="localhost",
    port=container_port
)

The next code executes the query in the database and loads the results and the name of the column.

In [4]:
with connection.cursor() as cursor:
    cursor.execute("SELECT 50 AS value1, 'test' AS value2;")
    columns = cursor.description
    data = cursor.fetchall()

print("Columns", columns)
print("Data", data)

Columns [<Column 'value1', type: int4 (oid: 23)>, <Column 'value2', type: text (oid: 25)>]
Data [(50, 'test')]


## No data

Some queries should not to return any data. You can check if the given query returnes any data by checking the `cursor.rowcount` attribute. If it takes `-1` value, it means that query doesn't return anything.

---

The following cell shows the `cursor.rowcount` attribute in the case of a `CREATE TABLE` query.

In [8]:
with connection.cursor() as cursor:
    cursor.execute("CREATE TABLE table1 (val1 INT, val2 INT);")
    rowcount = cursor.rowcount
    cursor.execute("DROP TABLE table1;")

rowcount

-1

As a result, `cursor.rowcount` takes the value `-1`. The next cell shows the same case but for the `SELECT` query to show difference.

In [7]:
with connection.cursor() as cursor:
    cursor.execute("SELECT 50;")
    rowcount = cursor.rowcount

rowcount

1